# ArcGIS Layer Summarizer

Uses the `ArcGISLayerStuffSummaryChain` and related classes to create a summary of an ArcGIS Feature Layer.

In [1]:
from langchain.chains.arcgis.row.base import ArcGISRowSummaryChain
from langchain.chains.arcgis.layer.base import ArcGISLayerStuffSummaryChain, ArcGISLayerSummaryInnerChain
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import ArcGISLoader
from langchain.document_transformers import ArcGISRowSummaryTransformer

In [2]:
%%time

url = "https://maps1.vcgov.org/arcgis/rest/services/Beaches/MapServer/7"

loader = ArcGISLoader(url, return_all_records=False, result_record_count=5)

docs = loader.load()

CPU times: user 1.22 s, sys: 155 ms, total: 1.37 s
Wall time: 3.6 s


In [3]:
llm = ChatOpenAI(temperature=0)

row_chain = ArcGISRowSummaryChain(llm=llm)

document_transformer = ArcGISRowSummaryTransformer(chain=row_chain)

In [4]:
import asyncio

import nest_asyncio

nest_asyncio.apply()

def main():

    async def _main():
        return await document_transformer.atransform_documents(docs)

    return asyncio.run(_main())

In [5]:
%%time

transformed_docs = main()

CPU times: user 66.2 ms, sys: 7.16 ms, total: 73.4 ms
Wall time: 5.99 s


In [6]:
layer_inner_chain = ArcGISLayerSummaryInnerChain(llm=llm)

stuff_chain = ArcGISLayerStuffSummaryChain(llm_chain=layer_inner_chain)

In [7]:
%%time

result = stuff_chain.combine_docs(transformed_docs)

CPU times: user 31.3 ms, sys: 0 ns, total: 31.3 ms
Wall time: 12.5 s


In [8]:
print(result[0])

Based on the summaries provided by the Beach Ramps API, here are the key findings:

1. There are multiple beach ramps located in different cities along the coast, including Ormond Beach, New Smyrna Beach, and Daytona Beach.

2. The beach ramps are identified by unique AccessIDs, such as OB-036, NS-110, NS-108, DB-043, and DB-042.

3. The beach ramps are classified as "OPEN VEHICLE RAMP," indicating that they are designed for vehicles to access the beach.

4. The specific locations of the beach ramps include CARDINAL DR in Ormond Beach, 600 block of S Atlantic Avenue in Ormond Beach, 500 BLK FLAGLER AV in New Smyrna Beach, 800 BLK N ATLANTIC AV in New Smyrna Beach, Hartford Avenue in Daytona Beach, and 2200 BLK N ATLANTIC AV in Daytona Beach.

5. The beach ramps have different milepost values, ranging from 2.19 to 12.76, indicating their positions along the coast.

6. The beach ramps are currently closed, as indicated by the "CLOSED" access status.

7. The entry date and time for the be